In [1]:
import pandas as pd
import hopsworks
import datetime
import matplotlib.pyplot as plt

import os
import sys
from dotenv import load_dotenv
from datetime import datetime, timedelta
import warnings
sys.path.append(os.path.abspath(os.path.join('..', 'functions')))
import util
sys.path.append(os.path.abspath(os.path.join('..', 'model')))
import torch
from sklearn.model_selection import train_test_split


warnings.filterwarnings("ignore")

In [2]:
load_dotenv()
proj = hopsworks.login()
fs = proj.get_feature_store("KTH_ID2223") 

2025-01-03 14:32:13,390 INFO: Initializing external client
2025-01-03 14:32:13,391 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-03 14:32:14,648 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164449


In [33]:
el_prices_fg = fs.get_feature_group(
    name='el_prices',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
power_fg = fs.get_feature_group(
    name='power',
    version=1,
)

In [34]:
el_prices_fg.show(10)
# weather_fg.show(10)
# power_fg.show(10)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.02s) 


,date,price
0,2023-01-23 00:00:00+00:00,1.662196
1,2023-12-20 00:00:00+00:00,0.535002
2,2023-05-27 00:00:00+00:00,0.064113
3,2023-03-15 00:00:00+00:00,0.975358
4,2022-11-01 00:00:00+00:00,0.655705
5,2023-07-27 00:00:00+00:00,0.381928
6,2023-01-26 00:00:00+00:00,1.101363
7,2024-06-02 00:00:00+00:00,0.097737
8,2023-03-22 00:00:00+00:00,0.488279
9,2024-09-04 00:00:00+00:00,0.209347


In [35]:
weather_df = weather_fg.read()
power_df = power_fg.read()
el_prices_df = el_prices_fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.80s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.68s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.67s) 


In [55]:
merged_df = (
    weather_df
    .merge(power_df, on="date", how="inner")
    .merge(el_prices_df, on="date", how="inner")
)

merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df = merged_df.sort_values(by="date").reset_index(drop=True)

dates = merged_df['date']
features = merged_df.drop(columns=['date', 'price']).iloc[:-1]
target = merged_df['price'].iloc[1:]


In [188]:
# X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=100)

For LSTM

In [7]:
# from sklearn.preprocessing import MinMaxScaler

# normalizer = MinMaxScaler()
# normalized_continuous_features = normalizer.fit_transform(features)
# normalized_features_df = pd.DataFrame(normalized_continuous_features, columns=features.columns)
# normalized_features_df

,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,sunshine_duration,hydro_mw,nuclear_mw,other_mw,wind_mw
0,0.653366,0.036923,0.261506,0.584641,0.000000,0.675887,0.706623,0.369720,0.208402
1,0.676117,0.027692,0.264039,0.521290,0.329079,0.588815,0.704314,0.371038,0.331940
2,0.631770,0.000000,0.315841,0.536078,0.423145,0.582232,0.703846,0.390193,0.380272
3,0.642915,0.000000,0.550507,0.403879,0.378280,0.463541,0.705697,0.363238,0.457186
4,0.625419,0.172308,0.474518,0.439128,0.000000,0.459842,0.706593,0.303004,0.489668
...,...,...,...,...,...,...,...,...,...
777,0.437348,0.000000,0.622481,0.793877,0.225706,0.591902,0.735926,0.340284,0.672421
778,0.443700,0.160000,0.571780,0.519946,0.000000,0.664265,0.735169,0.275513,0.616507
779,0.525927,0.064615,0.636439,0.656699,0.024690,0.631070,0.735428,0.172500,0.548422
780,0.413558,0.006154,0.418330,0.788535,0.000000,0.692749,0.736244,0.228868,0.635597


In [46]:
target

1      0.607735
2      0.550615
3      0.292413
4      0.422040
5      0.351532
         ...   
778    0.384965
779    0.230300
780    0.324242
781    0.075346
782    0.925848
Name: price, Length: 782, dtype: float64

Testing Lagged Prices for XGBoost

In [56]:
features['price_lag_1'] = target.shift(1).values
features['price_lag_2'] = target.shift(2).values
features['price_lag_3'] = target.shift(3).values
features

,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,sunshine_duration,hydro_mw,nuclear_mw,other_mw,wind_mw,price_lag_1,price_lag_2,price_lag_3
0,8.792083,1.2,13.202726,210.351074,0.000000,8805.291667,5699.208333,897.958333,2831.708333,NaN,NaN,NaN
1,9.612916,0.9,13.276144,187.686081,19597.978516,7981.458333,5689.541667,900.416667,4226.875000,0.607735,NaN,NaN
2,8.012917,0.0,14.777550,192.976761,25200.000000,7919.166667,5687.583333,936.125000,4772.708333,0.550615,0.607735,NaN
3,8.415000,0.0,21.578989,145.680145,22528.138672,6796.166667,5695.333333,885.875000,5641.333333,0.292413,0.550615,0.607735
4,7.783750,5.6,19.376562,158.291138,0.000000,6761.166667,5699.083333,773.583333,6008.166667,0.422040,0.292413,0.550615
...,...,...,...,...,...,...,...,...,...,...,...,...
777,0.998333,0.0,23.665035,285.209167,13441.726562,8010.666667,5821.875000,843.083333,8072.083333,0.735706,0.270241,0.273672
778,1.227500,5.2,22.195539,187.205170,0.000000,8695.333333,5818.708333,722.333333,7440.625000,0.384965,0.735706,0.270241
779,4.194167,2.1,24.069572,236.131195,1470.378784,8381.250000,5819.791667,530.291667,6671.708333,0.230300,0.384965,0.735706
780,0.140000,0.2,17.748037,283.297974,0.000000,8964.833333,5823.208333,635.375000,7656.208333,0.324242,0.230300,0.384965


In [58]:
# X = torch.from_numpy(normalized_features_df.values) # For LSTM
X = torch.from_numpy(features.values) # For XGBoost
y = torch.from_numpy(target.values)

train_num = int(0.80 * X.shape[0])
X_train = X[:train_num]
X_test = X[train_num:]
y_train = y[:train_num]
y_test = y[train_num:]

# val_num = int(0.2 * train_num)  
# X_val = X_train[-val_num:]
# y_val = y_train[-val_num:]
# X_train = X_train[:-val_num]
# y_train = y_train[:-val_num]


In [1134]:
X_train.size()


torch.Size([663, 9])

In [1135]:
input_features = [
        "temperature_2m_mean", "precipitation_sum", "wind_speed_10m_max",
        "wind_direction_10m_dominant", "sunshine_duration",
        "Hydro Water Reservoir", "Nuclear", "Other", "Wind Onshore", 
]
input_size = len(input_features)
# model = nn.EnergyPricePredictorLSTM(input_size=input_size)

In [1136]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
class SequenceDataset(Dataset):
    def __init__(self, X, y, seq_length):
        self.X = X
        self.y = y
        self.seq_length = seq_length

    def __len__(self):
        return len(self.X) // self.seq_length

    def __getitem__(self, idx):
        x_seq = self.X[idx:idx + self.seq_length]
        y_hist = self.y[idx:idx + self.seq_length].reshape(-1, 1) 
        x_combined = torch.tensor(np.hstack((x_seq, y_hist)), dtype=torch.float32)
        y_target = torch.tensor(self.y[idx + self.seq_length], dtype=torch.float32)
        return x_combined, y_target


        # y_seq = self.y[idx:idx + self.seq_length].unsqueeze(-1)
        # y_target = self.y[idx + self.seq_length]
        # return torch.tensor(y_seq, dtype=torch.float32), torch.tensor(y_target, dtype=torch.float32)



In [1137]:
seq_length = 3
input_size = 10
batch_size = 16
train_dataset = SequenceDataset(X_train, y_train, seq_length)
test_dataset = SequenceDataset(X_test, y_test, seq_length)

train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# val_dataset = SequenceDataset(X_val, y_val, seq_length)
# val_loader = DataLoader(val_dataset, batch_size=batch_size)


In [1138]:
for i, (X, y) in enumerate(train_loader):
    print(X.size())
    print(y.size())
    break

for i, (X, y) in enumerate(test_loader):
    print(X.size())
    print(y.size())
    break

torch.Size([16, 3, 10])
torch.Size([16])
torch.Size([16, 3, 10])
torch.Size([16])


In [1141]:
hidden_size = 16        
num_layers = 2     
output_size = 1

In [1142]:
import torch.optim as optim
import torch.nn as nn

class PriceLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(PriceLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p=0.4)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  
        out, _ = self.lstm(x, (h0, c0))  
        out = self.dropout(out)
        out = self.fc(out[:, -1, :]) 
        return out

num_epochs = 5000
model = PriceLSTM(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
train_loss_list = []
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    

    train_loss_list.append(train_loss/len(train_loader))
    if epoch % 25 == 0:
        print(f"Epoch {epoch+1}, Training Loss: {train_loss/len(train_loader):.4f}")

    # if epoch % 20 == 0:
    #     model.eval()

    #     val_loss = 0
    #     with torch.no_grad():
    #         for X_batch, y_batch in val_loader:
    #             y_pred = model(X_batch)
    #             val_loss += loss.item()
        
    #     print(f"Epoch {epoch+1}, Training Loss: {train_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(val_loader)}")

from sklearn.metrics import mean_squared_error, r2_score

model.eval()
test_loss = 0
y_preds = []
y_true = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        y_preds += y_pred.squeeze().tolist()
        y_true += y_batch.tolist()

mse = mean_squared_error(y_true, y_preds)
print("MSE:", mse)

r2 = r2_score(y_true, y_preds)
print("R squared:", r2)
# 0.15 0.12 0.24 / 3x64 
#      0.13          / 3x128
#      0.16          / 3x32
#      0.10   0.19    0.35 0.274  / 2x64
# 0.102 

Epoch 1, Training Loss: 2.3706
Epoch 26, Training Loss: 1.9691
Epoch 51, Training Loss: 1.2605
Epoch 76, Training Loss: 1.0397
Epoch 101, Training Loss: 0.8215
Epoch 126, Training Loss: 0.7150
Epoch 151, Training Loss: 0.5225
Epoch 176, Training Loss: 0.4641
Epoch 201, Training Loss: 0.4762
Epoch 226, Training Loss: 0.4390
Epoch 251, Training Loss: 0.3829
Epoch 276, Training Loss: 0.4037
Epoch 301, Training Loss: 0.2715
Epoch 326, Training Loss: 0.3298
Epoch 351, Training Loss: 0.3066
Epoch 376, Training Loss: 0.3095
Epoch 401, Training Loss: 0.3111
Epoch 426, Training Loss: 0.3111
Epoch 451, Training Loss: 0.2945
Epoch 476, Training Loss: 0.2328
Epoch 501, Training Loss: 0.2554
Epoch 526, Training Loss: 0.3115
Epoch 551, Training Loss: 0.3030
Epoch 576, Training Loss: 0.2928
Epoch 601, Training Loss: 0.2201
Epoch 626, Training Loss: 0.2258
Epoch 651, Training Loss: 0.2798
Epoch 676, Training Loss: 0.2145
Epoch 701, Training Loss: 0.2457
Epoch 726, Training Loss: 0.2237
Epoch 751, Trai

In [1117]:
target_variance = np.var(y_true)
print("Target Variance:", target_variance)


Target Variance: 0.012120836186714877


In [1118]:
target_mean = np.mean(y_true)
print("Target Mean:", target_mean)


Target Mean: 0.11867289241546622


In [1129]:
print(y_preds)
print(y_true)

[0.32315653562545776, 0.25238165259361267, 0.25038230419158936, 0.22440654039382935, 0.28922802209854126, 0.36193370819091797, 0.19421449303627014, 0.2526513338088989, 0.21887004375457764, 0.14466416835784912, 0.12356337904930115, 0.10380619019269943, 0.1399623602628708, 0.2858304977416992, 0.5071783661842346, 0.3986421823501587, 0.47549405694007874, 0.20019099116325378, 0.2068873792886734, 0.1932094246149063, 0.19209063053131104, 0.1959439516067505, 0.08877623826265335, 0.09596601128578186, 0.11146038770675659, 0.17437508702278137, 0.15514889359474182, 0.20192950963974, 0.17841124534606934, 0.3363396227359772, 0.13258112967014313, 0.16613391041755676, 0.19559583067893982, 0.29330918192863464, 0.22525247931480408, 0.4076455235481262, 0.4698227643966675, 0.4820707142353058, 0.2088499367237091]
[0.04659541696310043, 0.024965833872556686, 0.014926666393876076, 0.10545500367879868, 0.46143290400505066, 0.28074583411216736, 0.20934750139713287, 0.2059720903635025, 0.11346250027418137, 0.076

In [69]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.01)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

r2 = r2_score(y_test, y_pred)
print("R squared:", r2)

eval_dict = { 
    "MSE": str(mse),
    "R squared": str(r2),
}

MSE: 0.07444219519523172
R squared: 0.5720467092865618


In [70]:
print(y_pred.tolist())
print(y_test.tolist())

[0.16029027104377747, 0.14934468269348145, 0.23059038817882538, 0.2925604283809662, 0.23977287113666534, 0.254116028547287, 0.24189607799053192, 0.29992276430130005, 0.2171967774629593, 0.20159846544265747, 0.1929854303598404, 0.2896548807621002, 0.4172971546649933, 0.17731235921382904, 0.29140400886535645, 0.324682354927063, 0.3757933974266052, 0.13666345179080963, 0.4034905433654785, 0.21132895350456238, 0.21926088631153107, 0.08550919592380524, 0.15067371726036072, 0.16247326135635376, 0.260773241519928, 0.2945998013019562, 0.23177891969680786, 0.21089808642864227, 0.10155300796031952, 0.10985223203897476, 0.18600359559059143, 0.1226571798324585, 0.16855183243751526, 0.14357145130634308, 0.07849326729774475, 0.13393817842006683, 0.039084888994693756, 0.17184443771839142, 0.10186558216810226, 0.16711173951625824, 0.1531064510345459, 0.16869014501571655, 0.09945947676897049, 0.08234670758247375, 0.21550244092941284, 0.14611244201660156, 0.34786415100097656, 0.2542768120765686, 0.16390

Upload model to Hopsworks

In [71]:
model_dir = "price_prediction_model"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

In [72]:
xgb_model.save_model(model_dir + "/model.json")

In [73]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(features)
output_schema = Schema(target)

model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

mr = proj.get_model_registry()

price_prediction_model = mr.python.create_model(
    name="price_prediction_model", 
    metrics=eval_dict,
    model_schema=model_schema,
    input_example=target.sample().values, 
    description="Electricity Price Predicition Model",
)

price_prediction_model.save(model_dir)

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/3968394 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/20 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1162 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1164449/models/price_prediction_model/4


Model(name: 'price_prediction_model', version: 4)